# Ejercicio 50
Si decimos que un usuario sabe un idioma cuando tiene un nivel de babel mayor o
igual a 1 consiga un dataframe cuyas columnas son tipos de logs, el índice es la
cantidad de idiomas que sabe un usuario y las celdas la probabilidad de que esos
usuarios generen ese tipo de log.

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
dfLanguages = pd.read_csv('./WikipediaDataset/languages.csv')
dfLogs = pd.read_csv('./WikipediaDataset/logs.csv', usecols = ["contributor_id", "logtype"])

In [2]:
dfLogs

,contributor_id,logtype
0,2075.0,delete
1,2075.0,delete
2,2075.0,delete
3,7856.0,delete
4,7856.0,delete
...,...,...
12188941,4980693.0,create
12188942,561850.0,create
12188943,6335019.0,newusers
12188944,443260.0,create


## Limpiando y preparando los datasets
Primero se eliminan los logs hechos por contribuyentes no registrados.

In [3]:
dfLogs = dfLogs.loc[~dfLogs["contributor_id"].isnull(), :].reset_index(drop = True)
dfLogs

,contributor_id,logtype
0,2075.0,delete
1,2075.0,delete
2,2075.0,delete
3,7856.0,delete
4,7856.0,delete
...,...,...
12063688,4980693.0,create
12063689,561850.0,create
12063690,6335019.0,newusers
12063691,443260.0,create


In [4]:
dfLogs["logtype"].value_counts()

newusers             6217228
delete               2493224
create               1198242
move                  959966
block                 353760
patrol                257425
thanks                241947
renameuser            204441
protect               102744
upload                 15843
tag                     5290
eparticle               4805
rights                  3774
student                 2803
course                   990
massmessage              472
instructor               250
growthexperiments        141
import                   130
online                    82
campus                    42
institution               34
contentmodel              25
merge                     21
managetags                 5
gblrights                  4
usermerge                  2
gblblock                   1
Name: logtype, dtype: int64

Se cambia el nombre de la columna "contributor_id" a "id_usuario" para facilitar el merge. 

In [5]:
dfLogs.rename(columns = {'contributor_id': 'id_usuario'}, inplace = True)
dfLogs.head()

,id_usuario,logtype
0,2075.0,delete
1,2075.0,delete
2,2075.0,delete
3,7856.0,delete
4,7856.0,delete


In [6]:
dfLanguages

,babel_user,babel_lang,babel_level
0,4502458,FR,2
1,5928200,aa,0
2,46918,ab,0
3,2050449,ab,1
4,4715583,ace,0
...,...,...,...
35661,2089885,zh-Hant-HK,1
35662,2089885,zh-Hant-TW,3
35663,5036533,zu,0
35664,5928200,zu,0


Se procede a preparar el data frame de lenguajes. Como se necesita saber cuántos idiomas habla un determinado usuario, se cambia el nivel de lenguaje "N" de nativo a un número entero. Es importante que el número sea mayor a 1 ya que como es el lenguaje nativo, se supone que el usuario sabe muy bien el idioma. Se reemplaza así por el número 4.5 escogido arbitrariamente. 

In [7]:
dfLanguages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35666 entries, 0 to 35665
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   babel_user   35666 non-null  int64 
 1   babel_lang   35662 non-null  object
 2   babel_level  35666 non-null  object
dtypes: int64(1), object(2)
memory usage: 836.0+ KB


In [8]:
dfLanguages = dfLanguages[dfLanguages['babel_lang'].notna()]
dfLanguages["babel_level"] = dfLanguages["babel_level"].replace("N", 4.5)

In [9]:
dfLanguages.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35662 entries, 0 to 35665
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   babel_user   35662 non-null  int64 
 1   babel_lang   35662 non-null  object
 2   babel_level  35662 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.1+ MB


In [10]:
dfLanguages["babel_level"] = dfLanguages["babel_level"].astype(str).astype(float)
dfLanguages.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35662 entries, 0 to 35665
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   babel_user   35662 non-null  int64  
 1   babel_lang   35662 non-null  object 
 2   babel_level  35662 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.1+ MB


## Desarrollo

Como se necesitan saber si el usuario sabe o no el idioma, se toma como base (según enunciado) que un usuario sabe un idioma si su nivel es mayor o igual a 1. Se crea entonces
dicha columna.

In [11]:
dfLanguages["sabe_el_lenguaje"] = dfLanguages["babel_level"] >= 1
dfLanguages 

,babel_user,babel_lang,babel_level,sabe_el_lenguaje
0,4502458,FR,2.0,True
1,5928200,aa,0.0,False
2,46918,ab,0.0,False
3,2050449,ab,1.0,True
4,4715583,ace,0.0,False
...,...,...,...,...
35661,2089885,zh-Hant-HK,1.0,True
35662,2089885,zh-Hant-TW,3.0,True
35663,5036533,zu,0.0,False
35664,5928200,zu,0.0,False


Se obtienen la cantidad de idiomas que sabe cada usuario.

In [12]:
dfLanguages["cant_idiomas"] = dfLanguages.groupby("babel_user")["sabe_el_lenguaje"].transform("sum")
dfLanguages

,babel_user,babel_lang,babel_level,sabe_el_lenguaje,cant_idiomas
0,4502458,FR,2.0,True,3
1,5928200,aa,0.0,False,44
2,46918,ab,0.0,False,2
3,2050449,ab,1.0,True,7
4,4715583,ace,0.0,False,27
...,...,...,...,...,...
35661,2089885,zh-Hant-HK,1.0,True,10
35662,2089885,zh-Hant-TW,3.0,True,10
35663,5036533,zu,0.0,False,4
35664,5928200,zu,0.0,False,44


A partir de acá se tienen filas que repiten información. Por ejemplo si hay un usuario 203 que sabe inglés y español, hay una fila que tiene su nivel de español, y la cantidad de idiomas totales que sabe, y otra fila con su nivel de inglés, junto con la misma cantidad de idiomas totales que sabe. Para hacer el merge con el otro dataset es importante quedarse solo con las filas que aportan información nueva respecto a la cantidad de idiomas que sabe un usuario, por lo tanto se proceden a eliminar los duplicados. 

In [13]:
dfLanguages.drop_duplicates(subset = ["babel_user"], inplace = True, ignore_index = True)
dfLanguages

,babel_user,babel_lang,babel_level,sabe_el_lenguaje,cant_idiomas
0,4502458,FR,2.0,True,3
1,5928200,aa,0.0,False,44
2,46918,ab,0.0,False,2
3,2050449,ab,1.0,True,7
4,4715583,ace,0.0,False,27
...,...,...,...,...,...
10810,1795722,ru,4.5,True,1
10811,3472877,sr,4.5,True,1
10812,4237402,tr,4.5,True,1
10813,6114870,tr,4.5,True,1


Los idiomas (español, inglés, frances, etc) que sabe un usuario ya no son relevantes, solo es relevante la cantidad total de lenguajes que sabe cada uno, por lo que se procede a eliminar columnas con información innecesaria.

In [16]:
dfLanguages = dfLanguages.loc[:, ["babel_user", "cant_idiomas"]]
dfLanguages

,babel_user,cant_idiomas
0,4502458,3
1,5928200,44
2,46918,2
3,2050449,7
4,4715583,27
...,...,...
10810,1795722,1
10811,3472877,1
10812,4237402,1
10813,6114870,1


In [17]:
dfLanguages.value_counts()

babel_user  cant_idiomas
6332447     2               1
435429      3               1
436858      3               1
436780      1               1
436737      3               1
                           ..
1719001     3               1
1718857     3               1
1718263     3               1
1717644     7               1
30          9               1
Length: 10815, dtype: int64

Se cambia el nombre de la columna "babel_user" a "id_usuario" para facilitar el merge.

In [18]:
dfLanguages.rename(columns = {'babel_user': 'id_usuario'}, inplace = True)
dfLanguages.head()

,id_usuario,cant_idiomas
0,4502458,3
1,5928200,44
2,46918,2
3,2050449,7
4,4715583,27


In [19]:
dfMerge = pd.merge(dfLogs, dfLanguages, how = "left", on = ["id_usuario"])
dfMerge

,id_usuario,logtype,cant_idiomas
0,2075.0,delete,4.0
1,2075.0,delete,4.0
2,2075.0,delete,4.0
3,7856.0,delete,3.0
4,7856.0,delete,3.0
...,...,...,...
12063688,4980693.0,create,NaN
12063689,561850.0,create,2.0
12063690,6335019.0,newusers,NaN
12063691,443260.0,create,14.0


Como se hizo un merge "left" respecto al datagrame de logs, se mantienen todas las filas que tenía dicho dataframe. Puede suceder que haya usuarios que no tengan registro en el data set de los lenguajes, por lo que no se cuenta con información. Se reemplaza entonces los NaN con 0.

In [21]:
dfMerge['cant_idiomas'] = dfMerge['cant_idiomas'].fillna(0)
dfMerge = dfMerge.loc[:, ["logtype", "cant_idiomas"]]
dfMerge

,logtype,cant_idiomas
0,delete,4.0
1,delete,4.0
2,delete,4.0
3,delete,3.0
4,delete,3.0
...,...,...
12063688,create,0.0
12063689,create,2.0
12063690,newusers,0.0
12063691,create,14.0


In [29]:
dfCantIdiomasVsLogs = dfMerge.groupby("cant_idiomas")["logtype"].value_counts().unstack()
dfCantIdiomasVsLogs

logtype,block,campus,contentmodel,course,create,delete,eparticle,gblblock,gblrights,growthexperiments,...,online,patrol,protect,renameuser,rights,student,tag,thanks,upload,usermerge
cant_idiomas,,,,,,,,,,,,,,,,,,,,,
0.0,253750.0,12.0,20.0,691.0,908993.0,1672437.0,4132.0,1.0,1.0,141.0,...,33.0,137548.0,66467.0,199974.0,2113.0,2595.0,4837.0,163824.0,13095.0,2.0
1.0,1694.0,3.0,NaN,15.0,11336.0,1731.0,77.0,NaN,NaN,NaN,...,5.0,7702.0,669.0,NaN,44.0,21.0,120.0,5235.0,168.0,NaN
2.0,13922.0,NaN,NaN,48.0,116511.0,189966.0,101.0,NaN,NaN,NaN,...,8.0,36628.0,11009.0,205.0,378.0,21.0,104.0,24860.0,507.0,NaN
3.0,39032.0,20.0,NaN,157.0,59828.0,205024.0,313.0,NaN,NaN,NaN,...,20.0,25515.0,9738.0,84.0,207.0,67.0,110.0,17902.0,626.0,NaN
4.0,35516.0,NaN,5.0,39.0,44382.0,292138.0,83.0,NaN,3.0,NaN,...,4.0,31649.0,9887.0,3912.0,667.0,48.0,67.0,14240.0,812.0,NaN
5.0,2613.0,7.0,NaN,38.0,18082.0,37476.0,89.0,NaN,NaN,NaN,...,11.0,9014.0,1110.0,35.0,20.0,47.0,39.0,8479.0,264.0,NaN
6.0,3867.0,NaN,NaN,2.0,17275.0,52150.0,9.0,NaN,NaN,NaN,...,NaN,1821.0,2264.0,113.0,234.0,3.0,6.0,3953.0,122.0,NaN
7.0,3069.0,NaN,NaN,NaN,3730.0,35656.0,1.0,NaN,NaN,NaN,...,NaN,7535.0,1258.0,117.0,63.0,1.0,5.0,1066.0,95.0,NaN
8.0,NaN,NaN,NaN,NaN,2736.0,134.0,NaN,NaN,NaN,NaN,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,1258.0,75.0,NaN


Los NaN que aparecen significan que para una cantidad de idiomas x, no se registró ningún log del tipo y. Por ejemplo, ningún usuario que hablara 34 idiomas hizo un log del tipo block. Se procede a reemplazar esos NaNs con 0.

In [36]:
dfCantIdiomasVsLogs = dfCantIdiomasVsLogs.fillna(0)
dfCantIdiomasVsLogs

logtype,block,campus,contentmodel,course,create,delete,eparticle,gblblock,gblrights,growthexperiments,...,online,patrol,protect,renameuser,rights,student,tag,thanks,upload,usermerge
cant_idiomas,,,,,,,,,,,,,,,,,,,,,
0.0,253750.0,12.0,20.0,691.0,908993.0,1672437.0,4132.0,1.0,1.0,141.0,...,33.0,137548.0,66467.0,199974.0,2113.0,2595.0,4837.0,163824.0,13095.0,2.0
1.0,1694.0,3.0,0.0,15.0,11336.0,1731.0,77.0,0.0,0.0,0.0,...,5.0,7702.0,669.0,0.0,44.0,21.0,120.0,5235.0,168.0,0.0
2.0,13922.0,0.0,0.0,48.0,116511.0,189966.0,101.0,0.0,0.0,0.0,...,8.0,36628.0,11009.0,205.0,378.0,21.0,104.0,24860.0,507.0,0.0
3.0,39032.0,20.0,0.0,157.0,59828.0,205024.0,313.0,0.0,0.0,0.0,...,20.0,25515.0,9738.0,84.0,207.0,67.0,110.0,17902.0,626.0,0.0
4.0,35516.0,0.0,5.0,39.0,44382.0,292138.0,83.0,0.0,3.0,0.0,...,4.0,31649.0,9887.0,3912.0,667.0,48.0,67.0,14240.0,812.0,0.0
5.0,2613.0,7.0,0.0,38.0,18082.0,37476.0,89.0,0.0,0.0,0.0,...,11.0,9014.0,1110.0,35.0,20.0,47.0,39.0,8479.0,264.0,0.0
6.0,3867.0,0.0,0.0,2.0,17275.0,52150.0,9.0,0.0,0.0,0.0,...,0.0,1821.0,2264.0,113.0,234.0,3.0,6.0,3953.0,122.0,0.0
7.0,3069.0,0.0,0.0,0.0,3730.0,35656.0,1.0,0.0,0.0,0.0,...,0.0,7535.0,1258.0,117.0,63.0,1.0,5.0,1066.0,95.0,0.0
8.0,0.0,0.0,0.0,0.0,2736.0,134.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1258.0,75.0,0.0


Se divide cada incidencia con la cantidad de logs total hechos. Esto dará una probabilidad del tipo: mediciones favorables / mediciones totales.

In [33]:
cantLogsHechos = len(dfLogs)
cantLogsHechos

12063693

In [37]:
dfProbabilidades = dfCantIdiomasVsLogs / cantLogsHechos
dfProbabilidades

logtype,block,campus,contentmodel,course,create,delete,eparticle,gblblock,gblrights,growthexperiments,...,online,patrol,protect,renameuser,rights,student,tag,thanks,upload,usermerge
cant_idiomas,,,,,,,,,,,,,,,,,,,,,
0.0,0.021034,9.947203e-07,1.657867e-06,5.727931e-05,7.534948e-02,1.386339e-01,3.425153e-04,8.289336e-08,8.289336e-08,0.000012,...,2.735481e-06,0.011402,5.509673e-03,1.657652e-02,0.000175,2.151083e-04,4.009552e-04,1.357992e-02,1.085488e-03,1.657867e-07
1.0,0.000140,2.486801e-07,0.000000e+00,1.243400e-06,9.396791e-04,1.434884e-04,6.382788e-06,0.000000e+00,0.000000e+00,0.000000,...,4.144668e-07,0.000638,5.545566e-05,0.000000e+00,0.000004,1.740760e-06,9.947203e-06,4.339467e-04,1.392608e-05,0.000000e+00
2.0,0.001154,0.000000e+00,0.000000e+00,3.978881e-06,9.657988e-03,1.574692e-02,8.372229e-06,0.000000e+00,0.000000e+00,0.000000,...,6.631468e-07,0.003036,9.125730e-04,1.699314e-05,0.000031,1.740760e-06,8.620909e-06,2.060729e-03,4.202693e-05,0.000000e+00
3.0,0.003235,1.657867e-06,0.000000e+00,1.301426e-05,4.959344e-03,1.699513e-02,2.594562e-05,0.000000e+00,0.000000e+00,0.000000,...,1.657867e-06,0.002115,8.072155e-04,6.963042e-06,0.000017,5.553855e-06,9.118269e-06,1.483957e-03,5.189124e-05,0.000000e+00
4.0,0.002944,0.000000e+00,4.144668e-07,3.232841e-06,3.678973e-03,2.421630e-02,6.880149e-06,0.000000e+00,2.486801e-07,0.000000,...,3.315734e-07,0.002623,8.195666e-04,3.242788e-04,0.000055,3.978881e-06,5.553855e-06,1.180401e-03,6.730941e-05,0.000000e+00
5.0,0.000217,5.802535e-07,0.000000e+00,3.149948e-06,1.498878e-03,3.106511e-03,7.377509e-06,0.000000e+00,0.000000e+00,0.000000,...,9.118269e-07,0.000747,9.201163e-05,2.901267e-06,0.000002,3.895988e-06,3.232841e-06,7.028528e-04,2.188385e-05,0.000000e+00
6.0,0.000321,0.000000e+00,0.000000e+00,1.657867e-07,1.431983e-03,4.322889e-03,7.460402e-07,0.000000e+00,0.000000e+00,0.000000,...,0.000000e+00,0.000151,1.876706e-04,9.366949e-06,0.000019,2.486801e-07,4.973601e-07,3.276774e-04,1.011299e-05,0.000000e+00
7.0,0.000254,0.000000e+00,0.000000e+00,0.000000e+00,3.091922e-04,2.955646e-03,8.289336e-08,0.000000e+00,0.000000e+00,0.000000,...,0.000000e+00,0.000625,1.042798e-04,9.698523e-06,0.000005,8.289336e-08,4.144668e-07,8.836432e-05,7.874869e-06,0.000000e+00
8.0,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,2.267962e-04,1.110771e-05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,...,0.000000e+00,0.000000,1.657867e-07,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,1.042798e-04,6.217002e-06,0.000000e+00


Como es un dataframe con probabilidades, si se suman todas las filas y columnas debería dar 1.

In [38]:
dfProbabilidades.sum().sum()

0.999999834213287